In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
mnist = MNIST(download=True,root='/data/zhuyuanjin/Mnist')

In [ ]:
class MnistSet(Dataset):
    def __init__(self):
        return
    def __len__(self):
        return len(mnist)
    def __getitem__(self, idx):
        X = np.array(mnist[idx][0]).astype(np.float32)
        y = mnist[idx][1]
        return {'X':X, 'y':y}

In [ ]:
dataset = MnistSet()
dataloader = DataLoader(dataset, num_workers=4, batch_size=128)

In [ ]:
for batch in dataloader:
    X = batch['X']
    y = batch['y']
    break

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 512)
        self.fc2 = nn.Linear(512, 1024)
        self.fc3 = nn.Linear(1024, 2048)
        self.fc4 = nn.Linear(2048, 1024)
        self.fc5 = nn.Linear(1024,784)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = x.view(x.size(0), 28, 28)
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=4, stride=2)
        self.fc1 = nn.Linear(32, 16)
        self.head = nn.Linear(16, 2)
    def forward(self, x):
        x = x.unsqueeze(1)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.head(x)
        prob = F.softmax(x)
        return prob

In [ ]:
eps = 1e-10
G = Generator()
D = Discriminator()
optG = torch.optim.Adam(G.parameters())
optD = torch.optim.Adam(D.parameters())

In [ ]:
def trainD(batch):
    z = torch.rand(128, 1)
    X = batch['X']
    y = batch['y']
    loss = -((1-D(G(z))[:, 0] + eps).log().mean() + (D(X)[:, 0] + eps).log().mean())
    optD.zero_grad()
    loss.backward()
    optD.step()
    return D(G(z))[:, 0].mean().item()
    
def trainG():
    z = torch.rand(128, 1)
    loss = (1-D(G(z))[:, 0] + eps).log().mean()
    optG.zero_grad()
    loss.backward()
    optG.step()
    return D(G(z))[:, 0].mean().item()
    
    

In [ ]:
G = Generator()
D = Discriminator()
optG = torch.optim.Adam(G.parameters())
optD = torch.optim.Adam(D.parameters())
lossD = 100
for epoch in range(100):
    for batch in dataloader:
        lossD = trainD(batch)
        print(lossD)
        if lossD < 0.2:
            print('现在开始训练Generator')
            k = 0
            while True:
                k += 1
                loss = trainG()
                print(loss)
                if loss > 0.9:
                    print('现在开始训练Disciminator')
                    img = G(torch.rand(1, 1)).squeeze().data.numpy()
                    plt.imshow(img, cmap='gray')
                    plt.show()
                    break
                
  
            
    
        


In [ ]:
img = np.array(mnist[0][0])
plt.imshow(img,cmap='gray')

In [ ]:
img.min()

In [ ]:
plt.imshow(img)

In [ ]:
for i in range(100):
    j = 0
    while True:
        j += 1
        print(j)
        if j>i:
            break

In [ ]:
torch.log(torch.tensor([1e-10]))